In [ ]:
import os
import treetaggerwrapper

# Définir les chemins
tagger_dir = os.path.expanduser("~/treetagger")  # Dossier TreeTagger
param_file = os.path.expanduser("~/treetagger/english.par")  # Modèle anglais
input_file = os.path.expanduser("~/treetagger/data/that_adv.txt")  # Fichier d'entrée

# Vérifier que les fichiers existent
assert os.path.exists(param_file), "Le fichier de paramètres n'existe pas."
assert os.path.exists(input_file), "Le fichier d'entrée n'existe pas."

# Initialiser TreeTagger
tagger = treetaggerwrapper.TreeTagger(TAGDIR=tagger_dir, TAGPARFILE=param_file)

# Lire le fichier `that_adv.txt`
with open(input_file, "r", encoding="utf-8") as f:
    text = f.read().splitlines()

# Appliquer TreeTagger ligne par ligne
for line in text:
    if line.strip():  # Vérifier que la ligne n'est pas vide
        tags = tagger.tag_text(line)
        print("\nTexte :", line)
        print("Annotation :")
        for tag in tags:
            print("\t".join(tag.split("\t")))  # Affiche proprement


Texte : I didn’t think it would be that hard.
Annotation :
I	PP	I
didn’t	VVD	didn’t
think	VV	think
it	PP	it
would	MD	would
be	VB	be
that	RB	that
hard	JJ	hard
.	SENT	.

Texte : It wasn’t that expensive after all.
Annotation :
It	PP	it
wasn’t	VVD	wasn’t
that	IN/that	that
expensive	JJ	expensive
after	IN	after
all	DT	all
.	SENT	.

Texte : She’s never been that interested in sports.
Annotation :
She’s	NP	She’s
never	RB	never
been	VBN	be
that	IN/that	that
interested	JJ	interested
in	IN	in
sports	NNS	sport
.	SENT	.

Texte : He didn’t run that fast.
Annotation :
He	PP	he
didn’t	VVD	didn’t
run	VV	run
that	IN/that	that
fast	RB	fast
.	SENT	.

Texte : I didn’t know it was that far.
Annotation :
I	PP	I
didn’t	VVD	didn’t
know	VV	know
it	PP	it
was	VBD	be
that	IN/that	that
far	RB	far
.	SENT	.

Texte : It doesn’t hurt that much.
Annotation :
It	PP	it
doesn’t	VVD	doesn’t
hurt	VV	hurt
that	IN/that	that
much	RB	much
.	SENT	.

Texte : I wasn’t that tired last night.
Annotation :
I	PP	I
wasn’t	VVD	wasn’t
t

In [6]:
import os
import treetaggerwrapper

# Définir les chemins
tagger_dir = os.path.expanduser("~/treetagger")  # Dossier TreeTagger
param_file = os.path.expanduser("~/treetagger/english.par")  # Modèle anglais
input_file = os.path.expanduser("~/treetagger/data/that_adv.txt")  # Texte à analyser
output_file = os.path.expanduser("~/treetagger/data/that-annotation.txt")  # Fichier où stocker les résultats

# Vérifier l'existence des fichiers
assert os.path.exists(param_file), "Le fichier de paramètres n'existe pas."
assert os.path.exists(input_file), "Le fichier d'entrée n'existe pas."

# Initialiser TreeTagger
tagger = treetaggerwrapper.TreeTagger(TAGDIR=tagger_dir, TAGPARFILE=param_file)

# Lire le fichier `that_adv.txt`
with open(input_file, "r", encoding="utf-8") as f:
    text = f.read().splitlines()

# Ouvrir le fichier de sortie
with open(output_file, "w", encoding="utf-8") as out_f:
    total_that = 0
    correct_that = 0
    expected_tag = "RB"  # Modifier si nécessaire (adverbe)

    # Appliquer TreeTagger et enregistrer les annotations
    for line in text:
        words = line.strip().split()
        if words:
            tags = tagger.tag_text(line)
            for tagged_word in tags:
                parts = tagged_word.split("\t")
                if len(parts) == 3:
                    word, predicted_tag, lemma = parts
                    # Sauvegarder dans le fichier
                    out_f.write(f"{word}\t{predicted_tag}\t{lemma}\n")
                    
                    # Vérifier si "that" est bien annoté
                    if word.lower() == "that":
                        total_that += 1
                        if predicted_tag == expected_tag:
                            correct_that += 1

# Calculer la précision sur "that"
accuracy = correct_that / total_that if total_that > 0 else 0
print(f"Précision de TreeTagger sur 'that' : {accuracy:.2%}")
print(f"Total de 'that' trouvés : {total_that}, Correctement annotés : {correct_that}")

# Vérifier le fichier enregistré
print(f"\nLes annotations sont sauvegardées dans : {output_file}")

Précision de TreeTagger sur 'that' : 5.00%
Total de 'that' trouvés : 100, Correctement annotés : 5

Les annotations sont sauvegardées dans : /Users/diamouserignetoubandiaye/treetagger/data/that-annotation.txt


In [38]:
import subprocess
import os

# Définir les chemins
tagger_bin = os.path.expanduser("~/treetagger/bin/train-tree-tagger")  # Adapter selon l'OS
lexicon_file = "lexicon_brown.txt"
open_class_file = "tags_brown.txt"
train_file = "corpus_brown_that_custom.txt"
output_model = "english_brown_model.par"

# Vérifier que les fichiers existent
for file in [tagger_bin, lexicon_file, open_class_file, train_file]:
    if not os.path.exists(file):
        print(f"Erreur : Fichier introuvable -> {file}")
        exit(1)

# Commande correcte pour entraîner TreeTagger
train_command = [
    tagger_bin, "-utf8",
    "-st", ".",
    lexicon_file, open_class_file, train_file, output_model
]

# Exécuter l'entraînement
process = subprocess.run(train_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Afficher la sortie
print(process.stdout.decode())
print(process.stderr.decode())

# Vérifier si le fichier modèle a été généré
if os.path.exists(output_model):
    print(f"✅ Modèle entraîné avec succès : {output_model}")
else:
    print(f"❌ Erreur : le modèle n'a pas été généré.")



train-tree-tagger -cl 2 -dtg 1.00 -sw 1.00 -ecw 1.00 -stg 1.00 -ptg -1.00 lexicon_brown.txt tags_brown.txt corpus_brown_that_custom.txt english_brown_model.par

	reading the lexicon ...
		reading the tagset ...
		reading the lemmas ...
		reading the entries ...
		sorting the lexicon ...
		reading the open class tags ...
	calculating tag frequencies ...
247000	making affix tree ...
prefix lexicon: 15945 nodes
suffix lexicon: 3120 nodes
	reading classes ...
	making ngram table ...
279449	29083
finished.
	making decision tree ...
58	saving parameters ...

Number of nodes: 59
Max. path length: 24

done.

✅ Modèle entraîné avec succès : english_brown_model.par


In [10]:
import subprocess

# Définir les chemins
model_file = os.path.expanduser("~/treetagger/data/that_model.par")
test_file = os.path.expanduser("~/treetagger/data/that_adv.txt")  # Fichier à tester
output_file = os.path.expanduser("~/treetagger/data/that_test_output.txt")

# Commande pour exécuter TreeTagger avec le nouveau modèle
tagger_command = [
    "~/treetagger/bin/tree-tagger",
    model_file
]

# Lire le fichier de test et l'envoyer à TreeTagger
with open(test_file, "r", encoding="utf-8") as f:
    test_text = f.read()

process = subprocess.run(tagger_command, input=test_text.encode(), stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

# Sauvegarder la sortie dans un fichier
with open(output_file, "w", encoding="utf-8") as f:
    f.write(process.stdout.decode())

# Afficher la sortie
print("Résultats de l'annotation :")
print(process.stdout.decode())

Résultats de l'annotation :



In [11]:
import subprocess

# Définir le chemin du modèle entraîné
model_file = os.path.expanduser("~/treetagger/data/that_model.par")
test_file = os.path.expanduser("~/treetagger/data/that_adv.txt")  # Fichier à tester

# Commande pour exécuter TreeTagger avec le nouveau modèle
tagger_command = [
    "~/treetagger/bin/tree-tagger",
    model_file
]

# Lire le fichier de test
with open(test_file, "r", encoding="utf-8") as f:
    test_text = f.read()

# Exécuter TreeTagger et capturer la sortie
process = subprocess.run(tagger_command, input=test_text.encode(), stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

# Afficher les erreurs s'il y en a
if process.stderr:
    print("⚠ Erreur TreeTagger :")
    print(process.stderr.decode())

# Afficher les résultats
print("Résultats de l'annotation :")
print(process.stdout.decode())

⚠ Erreur TreeTagger :

USAGE:  tree-tagger {-options-} <parameter file> {<input file> {<output file>}}

OPTIONS:
	-token:	Print the token
	-lemma:	Print the lemma
	-sgml:	Don't tag SGML annotations
	-threshold <p>: Print all tags of a word with a probability higher than <p> times the largest probability
	-prob: Print tag probabilities
	-ignore-prefix: Ignore prefix when guessing pos for unknown words.
	-no-unknown: Print the token rather than <unknown> for unknown lemmas
	-hyphen-heuristics:	Turn on the heuristics fur guessing the parts of speech of unknown hyphenated words
	-quiet:	Don't print status messages
	-pt-with-lemma:	pretagging with lemmata
	-pt-with-prob:	pretagging with probabilities
	-files <f>:	Read names of input and output files pairwise from <f>
	-lex <f>:	Read auxiliary lexicon entries from file <f>
	-wc <f>:	Read a word-class automaton from file <f>
	-eos-tag <tag>:	The SGML tag <tag> signals the end of a sentence.
			This option implies the option -sgml

Some more e

In [12]:
import subprocess
import os

# Définir les chemins des fichiers
model_file = os.path.expanduser("~/treetagger/data/that_model.par")
test_file = os.path.expanduser("~/treetagger/data/that_adv.txt")  # Fichier de test
output_file = os.path.expanduser("~/treetagger/data/that_test_output.txt")  # Résultat

# Vérifier si le fichier modèle et le fichier test existent
assert os.path.exists(model_file), "⚠ Le modèle that_model.par n'existe pas !"
assert os.path.exists(test_file), "⚠ Le fichier de test that_adv.txt n'existe pas !"

# Commande correcte pour exécuter TreeTagger
tagger_command = [
    "~/treetagger/bin/tree-tagger",
    model_file,
    test_file
]

# Exécuter TreeTagger et récupérer la sortie
process = subprocess.run(" ".join(tagger_command), shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Sauvegarder la sortie dans un fichier
with open(output_file, "w", encoding="utf-8") as f:
    f.write(process.stdout.decode())

# Vérifier s'il y a une erreur
if process.stderr:
    print("⚠ Erreur TreeTagger :")
    print(process.stderr.decode())
else:
    print("✅ Annotation terminée ! Résultats enregistrés dans that_test_output.txt")

# Afficher les résultats directement
print("\nRésultats de l'annotation :")
print(process.stdout.decode())

⚠ Erreur TreeTagger :
	reading parameters ...
	tagging ...
	 finished.


Résultats de l'annotation :
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT
SENT



In [13]:
import subprocess
import os

# Définir le chemin du modèle
model_file = os.path.expanduser("~/treetagger/data/that_model.par")

# Tester une phrase directement en entrée
test_sentence = "I didn’t think it would be that hard.\n"

tagger_command = [
    "~/treetagger/bin/tree-tagger",
    model_file
]

process = subprocess.run(" ".join(tagger_command), input=test_sentence.encode(), stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

print("📌 Résultat de l'annotation :")
print(process.stdout.decode())

if process.stderr:
    print("⚠ Erreur TreeTagger :")
    print(process.stderr.decode())

📌 Résultat de l'annotation :
SENT

⚠ Erreur TreeTagger :
	reading parameters ...
	tagging ...
	 finished.



In [14]:
import os

model_file = os.path.expanduser("~/treetagger/data/that_model.par")

# Vérifier si le modèle existe et sa taille
if os.path.exists(model_file) and os.path.getsize(model_file) > 1000:  # Minimum 1 Ko
    print(f"✅ Le modèle existe et fait {os.path.getsize(model_file)} octets.")
else:
    print("⚠ Le modèle `that_model.par` est absent ou trop petit, l'entraînement a peut-être échoué.")

⚠ Le modèle `that_model.par` est absent ou trop petit, l'entraînement a peut-être échoué.


In [15]:
import subprocess

model_file = os.path.expanduser("~/treetagger/data/that_model.par")
test_file = os.path.expanduser("~/treetagger/data/that_adv.txt")

tagger_command = [
    "~/treetagger/bin/tree-tagger",
    "-token",  # Forcer l'affichage des tokens
    model_file,
    test_file
]

process = subprocess.run(" ".join(tagger_command), shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

print("📌 Résultat de l'annotation (avec -token) :")
print(process.stdout.decode())

if process.stderr:
    print("⚠ Erreur TreeTagger :")
    print(process.stderr.decode())

📌 Résultat de l'annotation (avec -token) :
I didn’t think it would be that hard.	SENT
It wasn’t that expensive after all.	SENT
She’s never been that interested in sports.	SENT
He didn’t run that fast.	SENT
I didn’t know it was that far.	SENT
It doesn’t hurt that much.	SENT
I wasn’t that tired last night.	SENT
The movie wasn’t that good.	SENT
She didn’t seem that upset about it.	SENT
It wasn’t that big of a deal.	SENT
The weather wasn’t that bad.	SENT
I didn’t realize the book was that old.	SENT
He’s not that tall compared to his brother.	SENT
I don’t think it’s that important.	SENT
The cake didn’t taste that sweet.	SENT
She wasn’t that impressed with the presentation.	SENT
The car wasn’t that expensive to fix.	SENT
He didn’t speak that loudly during the meeting.	SENT
I wasn’t that worried about the exam.	SENT
It wasn’t that late when I left.	SENT
The hotel wasn’t that luxurious, but it was nice.	SENT
He didn’t look that tired after the marathon.	SENT
It wasn’t that surprising to hear t

In [7]:
import os
def generate_lexicon(training_corpus_path, lexicon_output_path):
    """
    Génère un fichier lexique à partir d'un corpus d’entraînement annoté.
    
    Le corpus doit contenir des lignes au format :
      token<TAB>tag<TAB>lemme
    Les lignes qui ne contiennent pas exactement 3 colonnes (par exemple, les en-têtes)
    seront ignorées.
    
    Pour chaque token, on sélectionne l'analyse (tag et lemme) la plus fréquente.
    
    :param training_corpus_path: Chemin vers le corpus d’entraînement
    :param lexicon_output_path: Chemin de sortie pour le fichier lexique généré
    """
    lexicon = {}  # Dictionnaire : token -> {(tag, lemme): fréquence}

    with open(training_corpus_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue  # Ignore les lignes vides
            # Ignore les lignes qui ne contiennent pas de tabulations (en-têtes, etc.)
            if "\t" not in line:
                continue
            parts = line.split("\t")
            if len(parts) != 3:
                # Affiche un avertissement et passe à la ligne suivante
                print(f"Avertissement : ligne ignorée (format inattendu) : {line}")
                continue
            token, tag, lemma = parts[0], parts[1], parts[2]
            if token not in lexicon:
                lexicon[token] = {}
            key = (tag, lemma)
            lexicon[token][key] = lexicon[token].get(key, 0) + 1

    # Pour chaque token, on sélectionne l'analyse (tag, lemme) la plus fréquente
    with open(lexicon_output_path, "w", encoding="utf-8") as out:
        for token, analyses in lexicon.items():
            best_analysis = max(analyses.items(), key=lambda item: item[1])[0]
            best_tag, best_lemma = best_analysis
            out.write(f"{token}\t{best_tag}\t{best_lemma}\n")

    print(f"Fichier lexique généré : {lexicon_output_path}")

# Exemple d'utilisation :
if __name__ == "__main__":
    # Remplacez ces chemins par ceux de vos fichiers
    training_corpus = os.path.expanduser("~/treetagger/data/that-annotation.txt")
    lexicon_file = os.path.expanduser("~/treetagger/data/lexicon2.txt")
    generate_lexicon(training_corpus, lexicon_file)

Fichier lexique généré : /Users/diamouserignetoubandiaye/treetagger/data/lexicon2.txt


In [8]:
def generate_tagset(corpus_file_path, tagset_output_path):
    """
    Génère un fichier listant tous les tags uniques présents dans le corpus d'entraînement.
    
    On suppose que chaque ligne du corpus est au format :
      token<TAB>tag<TAB>lemme
    Les lignes non conformes (par exemple, les en-têtes) sont ignorées.
    
    :param corpus_file_path: Chemin vers le corpus d'entraînement annoté
    :param tagset_output_path: Chemin où écrire le fichier du tagset (liste des tags)
    """
    tags = set()
    
    with open(corpus_file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue  # ignorer les lignes vides
            # Vérifier que la ligne contient bien une tabulation
            if "\t" not in line:
                continue
            parts = line.split("\t")
            if len(parts) != 3:
                # Ligne au format inattendu, on l'ignore
                print(f"Ligne ignorée (format inattendu) : {line}")
                continue
            # La deuxième colonne correspond au tag
            tag = parts[1]
            tags.add(tag)
    
    # Écriture du fichier de tagset (un tag par ligne)
    with open(tagset_output_path, "w", encoding="utf-8") as out:
        for tag in sorted(tags):
            out.write(tag + "\n")
    
    print(f"Fichier tagset généré : {tagset_output_path}")


# Exemple d'utilisation
if __name__ == "__main__":
    corpus_path = os.path.expanduser("~/treetagger/data/that-annotation.txt")  # Remplacez par le chemin de votre corpus annoté
    tagset_path = os.path.expanduser("~/treetagger/data/tagset.txt")          # Chemin de sortie souhaité pour la liste des tags
    generate_tagset(corpus_path, tagset_path)

Fichier tagset généré : /Users/diamouserignetoubandiaye/treetagger/data/tagset.txt


In [ ]:

# Définir les chemins
tagger_dir = os.path.expanduser("~/treetagger")  # Dossier TreeTagger
param_file = os.path.expanduser("~/treetagger/english.par")  # Modèle anglais
input_file = os.path.expanduser("~/treetagger/data/that_adv.txt")  # Fichier d'entrée

In [28]:
import os
import subprocess

def train_treetagger(corpus_path, lexicon_path, open_class_path, output_model_path):
    """
    Entraîne un modèle TreeTagger à partir d'un corpus annoté.
    
    Usage (d'après l'aide de train-tree-tagger) :
      train-tree-tagger [options] <lexicon> <open class file> <input file> <output file>
    
    Paramètres :
      - corpus_path : chemin vers le corpus d'entraînement annoté (format : token<TAB>tag<TAB>lemme par ligne)
      - lexicon_path : chemin vers le fichier lexique généré à partir du corpus
      - open_class_path : chemin vers le fichier listant les mots open class (ou fichier vide)
      - output_model_path : chemin où sera sauvegardé le modèle entraîné
    """
    # Construction de la commande avec les arguments dans l'ordre attendu.
    cmd = [
        os.path.expanduser("~/treetagger/bin/train-tree-tagger"),
        lexicon_path,
        open_class_path,
        corpus_path,
        output_model_path
    ]
    
    print("Commande d'entraînement exécutée :")
    print(" ".join(cmd))
    
    # Exécution de la commande
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    # Vérification du retour
    if result.returncode != 0:
        print("Erreur lors de l'entraînement :")
        print(result.stderr)
    else:
        print("Entraînement terminé avec succès.")
        print(result.stdout)

if __name__ == "__main__":
    # Définissez les chemins adaptés à votre configuration
    corpus_path      = os.path.expanduser("~/treetagger/data/that_train.txt")      # Votre corpus annoté
    lexicon_path     = os.path.expanduser("~/treetagger/data/lexicon2.txt")        # Fichier lexique généré
    open_class_path  = os.path.expanduser("~/treetagger/data/tagset.txt")      # Fichier open class
    
    # Si vous n'avez pas encore de fichier open class, vous pouvez en créer un vide
    if not os.path.exists(open_class_path):
        with open(open_class_path, "w", encoding="utf-8") as f:
            # Vous pouvez également y ajouter manuellement la liste des mots open class si nécessaire.
            f.write("")  # Fichier vide pour l'instant
    
    output_model_path = os.path.expanduser("~/treetagger/myEnglish.par")   # Chemin de sortie pour le modèle entraîné
    
    train_treetagger(corpus_path, lexicon_path, open_class_path, output_model_path)

Commande d'entraînement exécutée :
/Users/diamouserignetoubandiaye/treetagger/bin/train-tree-tagger /Users/diamouserignetoubandiaye/treetagger/data/lexicon2.txt /Users/diamouserignetoubandiaye/treetagger/data/tagset.txt /Users/diamouserignetoubandiaye/treetagger/data/that_train.txt /Users/diamouserignetoubandiaye/treetagger/myEnglish.par
Erreur lors de l'entraînement :

train-tree-tagger -cl 2 -dtg 1.00 -sw 1.00 -ecw 1.00 -stg 1.00 -ptg -1.00 /Users/diamouserignetoubandiaye/treetagger/data/lexicon2.txt /Users/diamouserignetoubandiaye/treetagger/data/tagset.txt /Users/diamouserignetoubandiaye/treetagger/data/that_train.txt /Users/diamouserignetoubandiaye/treetagger/myEnglish.par

	reading the lexicon ...
		reading the tagset ...
		reading the lemmas ...
		reading the entries ...
		sorting the lexicon ...
		reading the open class tags ...
	calculating tag frequencies ...

ERROR: unknown tag <NNP> !




In [26]:
import os
import treetaggerwrapper

# Définir les chemins
tagger_dir = os.path.expanduser("~/treetagger")  # Dossier TreeTagger
param_file = os.path.expanduser("~/treetagger/myenglish.par")  # Modèle anglais
input_file = os.path.expanduser("~/treetagger/data/that_adv.txt")  # Fichier d'entrée

# Vérifier que les fichiers existent
assert os.path.exists(param_file), "Le fichier de paramètres n'existe pas."
assert os.path.exists(input_file), "Le fichier d'entrée n'existe pas."

# Initialiser TreeTagger
tagger = treetaggerwrapper.TreeTagger(TAGDIR=tagger_dir, TAGPARFILE=param_file)

# Lire le fichier `that_adv.txt`
with open(input_file, "r", encoding="utf-8") as f:
    text = f.read().splitlines()

# Appliquer TreeTagger ligne par ligne
for line in text:
    if line.strip():  # Vérifier que la ligne n'est pas vide
        tags = tagger.tag_text(line)
        print("\nTexte :", line)
        print("Annotation :")
        for tag in tags:
            print("\t".join(tag.split("\t")))  # Affiche proprement


Texte : I didn’t think it would be that hard.
Annotation :
I	PP	I
didn’t	VVD	didn’t
think	VV	think
it	PP	it
would	MD	would
be	VB	be
that	IN/that	that
hard	JJ	hard
.	SENT	.

Texte : It wasn’t that expensive after all.
Annotation :
It	PP	it
wasn’t	NN	wasn’t
that	IN/that	that
expensive	JJ	expensive
after	IN	after
all	DT	all
.	SENT	.

Texte : She’s never been that interested in sports.
Annotation :
She’s	NP	She’s
never	RB	never
been	VBN	be
that	IN/that	that
interested	JJ	interested
in	IN	in
sports	NNS	sport
.	SENT	.

Texte : He didn’t run that fast.
Annotation :
He	PP	he
didn’t	VVD	didn’t
run	VV	run
that	IN/that	that
fast	RB	fast
.	SENT	.

Texte : I didn’t know it was that far.
Annotation :
I	PP	I
didn’t	VVD	didn’t
know	VV	know
it	PP	it
was	VBD	be
that	IN/that	that
far	RB	far
.	SENT	.

Texte : It doesn’t hurt that much.
Annotation :
It	PP	it
doesn’t	VVD	doesn’t
hurt	VV	hurt
that	IN/that	that
much	RB	much
.	SENT	.

Texte : I wasn’t that tired last night.
Annotation :
I	PP	I
wasn’t	NN	wasn’

In [ ]:
import os
import treetaggerwrapper

# Définir les chemins
tagger_dir = os.path.expanduser("~/treetagger")  # Dossier TreeTagger
param_file = os.path.expanduser("~/treetagger/english.par")  # Modèle anglais
input_file = os.path.expanduser("~/treetagger/data/that_adv.txt")  # Texte à analyser
output_file = os.path.expanduser("~/treetagger/data/that-annotation.txt")  # Fichier où stocker les résultats

# Vérifier l'existence des fichiers
assert os.path.exists(param_file), "Le fichier de paramètres n'existe pas."
assert os.path.exists(input_file), "Le fichier d'entrée n'existe pas."

# Initialiser TreeTagger
tagger = treetaggerwrapper.TreeTagger(TAGDIR=tagger_dir, TAGPARFILE=param_file)

# Lire le fichier `that_adv.txt`
with open(input_file, "r", encoding="utf-8") as f:
    text = f.read().splitlines()

# Ouvrir le fichier de sortie
with open(output_file, "w", encoding="utf-8") as out_f:
    total_that = 0
    correct_that = 0
    expected_tag = "RB"  # Modifier si nécessaire (adverbe)

    # Appliquer TreeTagger et enregistrer les annotations
    for line in text:
        words = line.strip().split()
        if words:
            tags = tagger.tag_text(line)
            for tagged_word in tags:
                parts = tagged_word.split("\t")
                if len(parts) == 3:
                    word, predicted_tag, lemma = parts
                    # Sauvegarder dans le fichier
                    out_f.write(f"{word}\t{predicted_tag}\t{lemma}\n")
                    
                    # Vérifier si "that" est bien annoté
                    if word.lower() == "that":
                        total_that += 1
                        if predicted_tag == expected_tag:
                            correct_that += 1

# Calculer la précision sur "that"
accuracy = correct_that / total_that if total_that > 0 else 0
print(f"Précision de TreeTagger sur 'that' : {accuracy:.2%}")
print(f"Total de 'that' trouvés : {total_that}, Correctement annotés : {correct_that}")

# Vérifier le fichier enregistré
print(f"\nLes annotations sont sauvegardées dans : {output_file}")

In [27]:
import os
import treetaggerwrapper

# Définir les chemins
tagger_dir = os.path.expanduser("~/treetagger")  # Dossier TreeTagger
param_file = os.path.expanduser("~/treetagger/myenglish.par")  # Modèle anglais
input_file = os.path.expanduser("~/treetagger/data/that_adv.txt")  # Texte à analyser
output_file = os.path.expanduser("~/treetagger/data/mythat-annotation.txt")  # Fichier où stocker les résultats

# Vérifier l'existence des fichiers
assert os.path.exists(param_file), "Le fichier de paramètres n'existe pas."
assert os.path.exists(input_file), "Le fichier d'entrée n'existe pas."

# Initialiser TreeTagger
tagger = treetaggerwrapper.TreeTagger(TAGDIR=tagger_dir, TAGPARFILE=param_file)

# Lire le fichier `that_adv.txt`
with open(input_file, "r", encoding="utf-8") as f:
    text = f.read().splitlines()

# Ouvrir le fichier de sortie
with open(output_file, "w", encoding="utf-8") as out_f:
    total_that = 0
    correct_that = 0
    expected_tag = "RB"  # Modifier si nécessaire (adverbe)

    # Appliquer TreeTagger et enregistrer les annotations
    for line in text:
        words = line.strip().split()
        if words:
            tags = tagger.tag_text(line)
            for tagged_word in tags:
                parts = tagged_word.split("\t")
                if len(parts) == 3:
                    word, predicted_tag, lemma = parts
                    # Sauvegarder dans le fichier
                    out_f.write(f"{word}\t{predicted_tag}\t{lemma}\n")
                    
                    # Vérifier si "that" est bien annoté
                    if word.lower() == "that":
                        total_that += 1
                        if predicted_tag == expected_tag:
                            correct_that += 1

# Calculer la précision sur "that"
accuracy = correct_that / total_that if total_that > 0 else 0
print(f"Précision de TreeTagger sur 'that' : {accuracy:.2%}")
print(f"Total de 'that' trouvés : {total_that}, Correctement annotés : {correct_that}")

# Vérifier le fichier enregistré
print(f"\nLes annotations sont sauvegardées dans : {output_file}")

Précision de TreeTagger sur 'that' : 0.00%
Total de 'that' trouvés : 100, Correctement annotés : 0

Les annotations sont sauvegardées dans : /Users/diamouserignetoubandiaye/treetagger/data/mythat-annotation.txt
